In [11]:
import pandas as pd

In [12]:
# in another file I was getting a lot of errors while converting 'BEGIN_DATE_TIME", will fix it and continue in this file 
raw_df = pd.read_csv("data_raw/StormEvents_combined.csv", low_memory=False)

# Print column names to make sure we’re accessing the right one
print(raw_df.columns)

# Now try to print the first few values
print(raw_df["BEGIN_DATE_TIME"].head())

for col in raw_df.columns:
    if "BEGIN" in col:
        print(col)

Index(['BEGIN_YEARMONTH', 'BEGIN_DAY', 'BEGIN_TIME', 'END_YEARMONTH',
       'END_DAY', 'END_TIME', 'EPISODE_ID', 'EVENT_ID', 'STATE', 'STATE_FIPS',
       'YEAR', 'MONTH_NAME', 'EVENT_TYPE', 'CZ_TYPE', 'CZ_FIPS', 'CZ_NAME',
       'WFO', 'BEGIN_DATE_TIME', 'CZ_TIMEZONE', 'END_DATE_TIME',
       'INJURIES_DIRECT', 'INJURIES_INDIRECT', 'DEATHS_DIRECT',
       'DEATHS_INDIRECT', 'DAMAGE_PROPERTY', 'DAMAGE_CROPS', 'SOURCE',
       'MAGNITUDE', 'MAGNITUDE_TYPE', 'FLOOD_CAUSE', 'CATEGORY', 'TOR_F_SCALE',
       'TOR_LENGTH', 'TOR_WIDTH', 'TOR_OTHER_WFO', 'TOR_OTHER_CZ_STATE',
       'TOR_OTHER_CZ_FIPS', 'TOR_OTHER_CZ_NAME', 'BEGIN_RANGE',
       'BEGIN_AZIMUTH', 'BEGIN_LOCATION', 'END_RANGE', 'END_AZIMUTH',
       'END_LOCATION', 'BEGIN_LAT', 'BEGIN_LON', 'END_LAT', 'END_LON',
       'EPISODE_NARRATIVE', 'EVENT_NARRATIVE', 'DATA_SOURCE'],
      dtype='object')
0    29-OCT-96 17:10:00
1    21-OCT-96 09:00:00
2    20-OCT-96 19:15:00
3    23-JAN-96 04:00:00
4    17-JAN-96 09:00:00
Name: BEGIN_

In [13]:
weather_df = pd.read_csv("data_raw/StormEvents_combined.csv", low_memory=False)

# Convert the BEGIN_DATE_TIME column using a specified format
weather_df["BEGIN_DATE_TIME"] = pd.to_datetime(
    weather_df["BEGIN_DATE_TIME"],
    format="%d-%b-%y %H:%M:%S",  #this matches '29-OCT-96 17:10:00'
    errors="coerce"
)

# Now check if it worked
print(weather_df["BEGIN_DATE_TIME"].head())
print(weather_df["BEGIN_DATE_TIME"].isna().sum())  # How many still failed to parse


0   1996-10-29 17:10:00
1   1996-10-21 09:00:00
2   1996-10-20 19:15:00
3   1996-01-23 04:00:00
4   1996-01-17 09:00:00
Name: BEGIN_DATE_TIME, dtype: datetime64[ns]
0


In [14]:
# Drop columns where more than 50% of values are NaN
threshold = len(weather_df) * 0.5
weather_df = weather_df.dropna(axis=1, thresh=threshold)
summary = weather_df.groupby(weather_df["BEGIN_DATE_TIME"].dt.date).agg({
    "DAMAGE_PROPERTY": "sum", 
    "EVENT_TYPE": "count"
}).reset_index()
# Save the full cleaned dataset
weather_df.to_csv("data_clean/weather_cleaned.csv", index=False)

# Save your daily summary
summary.to_csv("data_clean/noaa_daily_summary.csv", index=False)


In [15]:
weather_df.head()


,BEGIN_YEARMONTH,BEGIN_DAY,BEGIN_TIME,END_YEARMONTH,END_DAY,END_TIME,EPISODE_ID,EVENT_ID,STATE,STATE_FIPS,...,MAGNITUDE,BEGIN_LOCATION,END_LOCATION,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,EPISODE_NARRATIVE,EVENT_NARRATIVE,DATA_SOURCE
0,199610,29,1710,199610,30,600,2052149.0,5570068,ILLINOIS,17.0,...,52.0,NaN,NaN,NaN,NaN,NaN,NaN,Strong gradient winds followed behind a line o...,NaN,PDC
1,199610,21,900,199610,21,900,2052748.0,5570362,OKLAHOMA,40.0,...,NaN,DURANT,DURANT,NaN,NaN,NaN,NaN,Flash flood waters washed out a small bridge a...,NaN,PDC
2,199610,20,1915,199610,20,1915,2052749.0,5570363,OKLAHOMA,40.0,...,4.5,EMPIRE CITY,EMPIRE CITY,34.42,-98.03,34.42,-98.03,Hail as large as softballs broke out car and t...,NaN,PDC
3,199601,23,400,199601,24,2200,2404440.0,5537534,CALIFORNIA,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A storm that originated in the Gulf of Alaska ...,NaN,PDC
4,199601,17,900,199601,18,1800,2404601.0,5538413,MINNESOTA,27.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A major winter storm brought blizzard conditio...,NaN,PDC
